In [1]:
%load_ext autoreload

In [2]:
%aimport model

In [3]:
%aimport data

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import skorch

In [5]:
import numpy as np

In [6]:
import keras

Using TensorFlow backend.


In [7]:
import torch

In [8]:
dataset = "mnist"

In [9]:
dataset_config = data.DATASET_CONFIGS[dataset]

In [10]:
net = skorch.net.NeuralNetClassifier(
    module=model.WideResNet(
        dataset,
        dataset_config['size'],
        dataset_config['channels'],
        dataset_config['classes'],
        total_block_number=6,
        widen_factor=10,
    ),
    batch_size=32,
    max_epochs=10,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/wrn{last_epoch[epoch]}.pt", monitor=None))
    ],
    device='cuda',
    train_split=None, # Workaround for https://github.com/dnouri/skorch/issues/218
    lr=1e-04,
)
net.initialize()

<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=WideResNet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ResidualBlockGroup(
        (sequential): Sequential(
          (0): ResidualBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv_transform): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (1): ResidualBlock(
            (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [11]:
def process(images):
    pad_images = np.pad(images, ((0, 0), (2, 2), (2, 2)), 'constant')
    number_of_images, height, width = pad_images.shape
    return pad_images.reshape(number_of_images, 1, height, width).astype(np.float32)  / 255.0

In [12]:
(x_train_bytes, y_train_bytes), (x_test_bytes, y_test_bytes) = keras.datasets.fashion_mnist.load_data()
x_train = process(x_train_bytes)
x_test = process(x_test_bytes)
y_train = y_train_bytes.astype(np.int64)
y_test = y_test_bytes.astype(np.int64)

In [13]:
net.fit_loop(x_train, y_train)

/mnt/project-root/model.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.f(x)


Checkpoint! Saving model to models/wrn1.pt.
  epoch    train_loss       dur
-------  ------------  --------
      1        0.4006  125.9197
Checkpoint! Saving model to models/wrn2.pt.
      2        0.2326  126.5087
Checkpoint! Saving model to models/wrn3.pt.
      3        0.1782  127.1159
Checkpoint! Saving model to models/wrn4.pt.
      4        0.1348  126.9946
Checkpoint! Saving model to models/wrn5.pt.
      5        0.0993  127.0145
Checkpoint! Saving model to models/wrn6.pt.
      6        0.0736  126.2018
Checkpoint! Saving model to models/wrn7.pt.
      7        0.0575  126.5998
Checkpoint! Saving model to models/wrn8.pt.
      8        0.0455  126.9306
Checkpoint! Saving model to models/wrn9.pt.
      9        0.0371  126.8832
Checkpoint! Saving model to models/wrn10.pt.
     10        0.0326  126.8877


<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=WideResNet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ResidualBlockGroup(
        (sequential): Sequential(
          (0): ResidualBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv_transform): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (1): ResidualBlock(
            (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [14]:
prediction_test = net.predict(x_test)

/mnt/project-root/model.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.f(x)


In [16]:
(prediction_test == y_test).mean()

0.9255